# 3 Pointers Made against advanced_totals.csv

### Import packages

In [56]:
import os
import numpy as np
import pandas as pd
%matplotlib inline
from scipy.stats import pearsonr, zscore
import itertools

pd.set_option("display.max_columns", None)

### Set working directory

In [57]:
# Print working directory
cwd = os.getcwd()
print(f'Directory: {cwd}')

# Change working directory
os.chdir('/Users/tyler/OneDrive/Documents/Python/NBA')

# Print working directory
cwd = os.getcwd()
print(f'Directory: {cwd}')

Directory: C:\Users\tyler\OneDrive\Documents\Python\NBA
Directory: C:\Users\tyler\OneDrive\Documents\Python\NBA


## Exploratory Data Analysis

### Import data

In [58]:
advanced_df = pd.read_csv('backend/data/totals/advanced_totals.csv').drop(['Unnamed: 0'], axis=1)
shooting_df = pd.read_csv('backend/data/totals/game_totals.csv').drop(['Unnamed: 0'], axis=1)
shooting_df = shooting_df[['date', 'visitor', 'home', 'team', '3p']]

In [59]:
# Merge dataframes to have target variable
df = pd.merge(shooting_df, advanced_df, 
              left_on=['date', 'visitor', 'home', 'team'], right_on=['date', 'visitor', 'home', 'team'],
              how='left')

### Basic exploration

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36248 entries, 0 to 36247
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      36248 non-null  object 
 1   visitor   36248 non-null  object 
 2   home      36248 non-null  object 
 3   team      36248 non-null  int64  
 4   3p        36248 non-null  int64  
 5   ts_perc   36248 non-null  float64
 6   efg_perc  36248 non-null  float64
 7   3par      36248 non-null  float64
 8   ftr       36248 non-null  float64
 9   orb_perc  36248 non-null  float64
 10  drb_perc  36248 non-null  float64
 11  trb_perc  36248 non-null  float64
 12  ast_perc  36248 non-null  float64
 13  stl_perc  36248 non-null  float64
 14  blk_perc  36248 non-null  float64
 15  tov_perc  36248 non-null  float64
 16  usg_perc  36248 non-null  float64
 17  ortg      36248 non-null  float64
 18  drtg      36248 non-null  float64
dtypes: float64(14), int64(2), object(3)
memory usage: 5.5+ MB


In [61]:
# Convert 'date' column to Date object
df['date'] = pd.to_datetime(df['date'])

# Convert 'team' column to Team Name
df['team'] = np.where(df['team'], df['home'], df['visitor'])

# Rename target variable
df = df.rename({'3p': 'target'}, axis=1)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36248 entries, 0 to 36247
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      36248 non-null  datetime64[ns]
 1   visitor   36248 non-null  object        
 2   home      36248 non-null  object        
 3   team      36248 non-null  object        
 4   target    36248 non-null  int64         
 5   ts_perc   36248 non-null  float64       
 6   efg_perc  36248 non-null  float64       
 7   3par      36248 non-null  float64       
 8   ftr       36248 non-null  float64       
 9   orb_perc  36248 non-null  float64       
 10  drb_perc  36248 non-null  float64       
 11  trb_perc  36248 non-null  float64       
 12  ast_perc  36248 non-null  float64       
 13  stl_perc  36248 non-null  float64       
 14  blk_perc  36248 non-null  float64       
 15  tov_perc  36248 non-null  float64       
 16  usg_perc  36248 non-null  float64       
 17  ortg      36

In [62]:
# Return ten lastest dates team played
def last_10_date(team, date):
    schedule = df[df['team'] == team].sort_values(by='date').reset_index()
    date_index = schedule[schedule['date'] == date].index[0]
    if date_index - 10 < 0:
        return None, None, None, None, None, None, None, None, None, None
    else:
        date_1, date_2 = schedule.iloc[date_index - 1]['date'], schedule.iloc[date_index - 2]['date']
        date_3, date_4 = schedule.iloc[date_index - 3]['date'], schedule.iloc[date_index - 4]['date']
        date_5, date_6 = schedule.iloc[date_index - 5]['date'], schedule.iloc[date_index - 6]['date']
        date_7, date_8 = schedule.iloc[date_index - 7]['date'], schedule.iloc[date_index - 8]['date']
        date_9, date_10 = schedule.iloc[date_index - 9]['date'], schedule.iloc[date_index - 10]['date']
        return date_1, date_2, date_3, date_4, date_5, date_6, date_7, date_8, date_9, date_10

df['dates'] = df.apply(lambda x: last_10_date(x.team, x.date), axis=1)
df['date_1'], df['date_2'] = df['dates'].apply(lambda x: x[0]), df['dates'].apply(lambda x: x[1])
df['date_3'], df['date_4'] = df['dates'].apply(lambda x: x[2]), df['dates'].apply(lambda x: x[3])
df['date_5'], df['date_6'] = df['dates'].apply(lambda x: x[4]), df['dates'].apply(lambda x: x[5])
df['date_7'], df['date_8'] = df['dates'].apply(lambda x: x[6]), df['dates'].apply(lambda x: x[7])
df['date_9'], df['date_10'] = df['dates'].apply(lambda x: x[8]), df['dates'].apply(lambda x: x[9])

# X and y column names to merge on
y_cols = df.columns
x_cols = ['date', 'team', 'ts_perc', 'efg_perc', '3par', 'ftr', 
          'orb_perc', 'drb_perc', 'trb_perc', 'ast_perc', 'stl_perc', 'blk_perc',
          'tov_perc', 'usg_perc', 'ortg', 'drtg']

last_10_games = df[y_cols]
X = df[x_cols]

# Dataframe of target (3pt made by each team) and of variables (last 5 games stats for each team)
weeks = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10']
for week in weeks:
    last_10_games = pd.merge(last_10_games, X, left_on=['date' + week, 'team'], right_on=['date', 'team'], how='left', suffixes=('', week))

drop_cols = ['ts_perc', 'efg_perc', '3par', 'ftr', 'orb_perc', 'drb_perc', 'trb_perc', 'ast_perc', 'stl_perc', 'blk_perc',
             'tov_perc', 'usg_perc', 'ortg', 'drtg', 'dates', 'date_1', 'date_2', 'date_3', 'date_4', 'date_5', 
             'date_6', 'date_7', 'date_8', 'date_9', 'date_10']
last_10_games = last_10_games.drop(drop_cols, axis=1).dropna(axis=0)

# Take average of last 10 games
stats = ['ts_perc', 'efg_perc', '3par', 'ftr', 'orb_perc', 'drb_perc', 'trb_perc', 
         'ast_perc', 'stl_perc', 'blk_perc', 'tov_perc', 'usg_perc', 'ortg', 'drtg']
for stat in stats:
    last_10_games[stat] = 0
    for week in weeks:
        last_10_games[stat] = last_10_games[stat] + last_10_games[stat + week]
    
    last_10_games[stat] = last_10_games[stat] / len(weeks)

drop_cols = [tup[0] + tup[1] for tup in list(itertools.product(stats, weeks))]
last_10_games = last_10_games.drop(drop_cols, axis=1)

# Totals for target and variables
last_10_games = last_10_games.groupby(['date', 'visitor', 'home']).aggregate(
    {'target': 'sum', 'ts_perc': 'mean', 'efg_perc': 'mean', '3par': 'mean', 'ftr': 'mean', 'orb_perc': 'mean', 
     'drb_perc': 'mean', 'trb_perc': 'mean', 'ast_perc': 'mean', 'stl_perc': 'mean', 'blk_perc': 'mean', 
     'tov_perc': 'mean', 'usg_perc': 'mean', 'ortg': 'sum', 'drtg': 'sum'})

# Dataframe of team's last 15 performances

In [63]:
# Return ten lastest dates team played
def last_15_date(team, date):
    schedule = df[df['team'] == team].sort_values(by='date').reset_index()
    date_index = schedule[schedule['date'] == date].index[0]
    if date_index - 15 < 0:
        return None, None, None, None, None, None, None, None, None, None, None, None, None, None, None
    else:
        date_1, date_2 = schedule.iloc[date_index - 1]['date'], schedule.iloc[date_index - 2]['date']
        date_3, date_4 = schedule.iloc[date_index - 3]['date'], schedule.iloc[date_index - 4]['date']
        date_5, date_6 = schedule.iloc[date_index - 5]['date'], schedule.iloc[date_index - 6]['date']
        date_7, date_8 = schedule.iloc[date_index - 7]['date'], schedule.iloc[date_index - 8]['date']
        date_9, date_10 = schedule.iloc[date_index - 9]['date'], schedule.iloc[date_index - 10]['date']
        date_11, date_12 = schedule.iloc[date_index - 11]['date'], schedule.iloc[date_index - 12]['date']
        date_13, date_14 = schedule.iloc[date_index - 13]['date'], schedule.iloc[date_index - 14]['date']
        date_15 = schedule.iloc[date_index - 15]['date']
        return date_1, date_2, date_3, date_4, date_5, date_6, date_7, date_8, date_9, date_10, date_11, date_12, date_13, date_14, date_15

df['dates'] = df.apply(lambda x: last_15_date(x.team, x.date), axis=1)
df['date_1'], df['date_2'] = df['dates'].apply(lambda x: x[0]), df['dates'].apply(lambda x: x[1])
df['date_3'], df['date_4'] = df['dates'].apply(lambda x: x[2]), df['dates'].apply(lambda x: x[3])
df['date_5'], df['date_6'] = df['dates'].apply(lambda x: x[4]), df['dates'].apply(lambda x: x[5])
df['date_7'], df['date_8'] = df['dates'].apply(lambda x: x[6]), df['dates'].apply(lambda x: x[7])
df['date_9'], df['date_10'] = df['dates'].apply(lambda x: x[8]), df['dates'].apply(lambda x: x[9])
df['date_11'], df['date_12'] = df['dates'].apply(lambda x: x[10]), df['dates'].apply(lambda x: x[11])
df['date_13'], df['date_14'] = df['dates'].apply(lambda x: x[12]), df['dates'].apply(lambda x: x[13])
df['date_15'] = df['dates'].apply(lambda x: x[14])

In [64]:
# X and y column names to merge on
y_cols = df.columns
x_cols = ['date', 'team', 'ts_perc', 'efg_perc', '3par', 'ftr', 
          'orb_perc', 'drb_perc', 'trb_perc', 'ast_perc', 'stl_perc', 'blk_perc',
          'tov_perc', 'usg_perc', 'ortg', 'drtg']

last_15_games = df[y_cols]
X = df[x_cols]

# Dataframe of target (3pt made by each team) and of variables (last 15 games stats for each team)
dates_15 = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
for date in dates_15:
    last_15_games = pd.merge(last_15_games, X, left_on=['date' + date, 'team'], right_on=['date', 'team'], how='left', suffixes=('', date))

## Feature Engineering

In [65]:
# Create season variable
def season(month, year):
    if (month in list(range(10, 13)) and year == 2006) or (month in list(range(1, 7)) and year == 2007):
        return 2006
    elif (month in list(range(10, 13)) and year == 2007) or (month in list(range(1, 7)) and year == 2008):
        return 2007
    elif (month in list(range(10, 13)) and year == 2008) or (month in list(range(1, 7)) and year == 2009):
        return 2008
    elif (month in list(range(10, 13)) and year == 2009) or (month in list(range(1, 7)) and year == 2010):
        return 2009
    elif (month in list(range(10, 13)) and year == 2010) or (month in list(range(1, 7)) and year == 2011):
        return 2010
    elif (month in list(range(10, 11)) and year == 2011) or (month in list(range(1, 7)) and year == 2012):
        return 2011
    elif (month in list(range(10, 13)) and year == 2012) or (month in list(range(1, 7)) and year == 2013):
        return 2012
    elif (month in list(range(10, 13)) and year == 2013) or (month in list(range(1, 7)) and year == 2014):
        return 2013
    elif (month in list(range(10, 13)) and year == 2014) or (month in list(range(1, 7)) and year == 2015):
        return 2014
    elif (month in list(range(10, 13)) and year == 2015) or (month in list(range(1, 7)) and year == 2016):
        return 2015
    elif (month in list(range(10, 13)) and year == 2016) or (month in list(range(1, 7)) and year == 2017):
        return 2016
    elif (month in list(range(10, 13)) and year == 2017) or (month in list(range(1, 7)) and year == 2018):
        return 2017
    elif (month in list(range(10, 13)) and year == 2018) or (month in list(range(1, 7)) and year == 2019):
        return 2018
    elif (month in list(range(10, 13)) and year == 2019) or (month in list(range(1, 11)) and year == 2020):
        return 2019
    elif (month in list(range(12, 13)) and year == 2020) or (month in list(range(1, 8)) and year == 2021):
        return 2020

df['season'] = df['date'].apply(lambda x: season(x.month, x.year))
last_15_games['season'] = last_15_games['date'].apply(lambda x: season(x.month, x.year))
last_15_games.sample()

,date,visitor,home,team,target,ts_perc,efg_perc,3par,ftr,orb_perc,drb_perc,trb_perc,ast_perc,stl_perc,blk_perc,tov_perc,usg_perc,ortg,drtg,dates,date_1,date_2,date_3,date_4,date_5,date_6,date_7,date_8,date_9,date_10,date_11,date_12,date_13,date_14,date_15,date_1,ts_perc_1,efg_perc_1,3par_1,ftr_1,orb_perc_1,drb_perc_1,trb_perc_1,ast_perc_1,stl_perc_1,blk_perc_1,tov_perc_1,usg_perc_1,ortg_1,drtg_1,date_2,ts_perc_2,efg_perc_2,3par_2,ftr_2,orb_perc_2,drb_perc_2,trb_perc_2,ast_perc_2,stl_perc_2,blk_perc_2,tov_perc_2,usg_perc_2,ortg_2,drtg_2,date_3,ts_perc_3,efg_perc_3,3par_3,ftr_3,orb_perc_3,drb_perc_3,trb_perc_3,ast_perc_3,stl_perc_3,blk_perc_3,tov_perc_3,usg_perc_3,ortg_3,drtg_3,date_4,ts_perc_4,efg_perc_4,3par_4,ftr_4,orb_perc_4,drb_perc_4,trb_perc_4,ast_perc_4,stl_perc_4,blk_perc_4,tov_perc_4,usg_perc_4,ortg_4,drtg_4,date_5,ts_perc_5,efg_perc_5,3par_5,ftr_5,orb_perc_5,drb_perc_5,trb_perc_5,ast_perc_5,stl_perc_5,blk_perc_5,tov_perc_5,usg_perc_5,ortg_5,drtg_5,date_6,ts_perc_6,efg_perc_6,3par_6,ftr_6,orb_perc_6,drb_perc_6,trb_perc_6,ast_perc_6,stl_perc_6,blk_perc_6,tov_perc_6,usg_perc_6,ortg_6,drtg_6,date_7,ts_perc_7,efg_perc_7,3par_7,ftr_7,orb_perc_7,drb_perc_7,trb_perc_7,ast_perc_7,stl_perc_7,blk_perc_7,tov_perc_7,usg_perc_7,ortg_7,drtg_7,date_8,ts_perc_8,efg_perc_8,3par_8,ftr_8,orb_perc_8,drb_perc_8,trb_perc_8,ast_perc_8,stl_perc_8,blk_perc_8,tov_perc_8,usg_perc_8,ortg_8,drtg_8,date_9,ts_perc_9,efg_perc_9,3par_9,ftr_9,orb_perc_9,drb_perc_9,trb_perc_9,ast_perc_9,stl_perc_9,blk_perc_9,tov_perc_9,usg_perc_9,ortg_9,drtg_9,date_10,ts_perc_10,efg_perc_10,3par_10,ftr_10,orb_perc_10,drb_perc_10,trb_perc_10,ast_perc_10,stl_perc_10,blk_perc_10,tov_perc_10,usg_perc_10,ortg_10,drtg_10,date_11,ts_perc_11,efg_perc_11,3par_11,ftr_11,orb_perc_11,drb_perc_11,trb_perc_11,ast_perc_11,stl_perc_11,blk_perc_11,tov_perc_11,usg_perc_11,ortg_11,drtg_11,date_12,ts_perc_12,efg_perc_12,3par_12,ftr_12,orb_perc_12,drb_perc_12,trb_perc_12,ast_perc_12,stl_perc_12,blk_perc_12,tov_perc_12,usg_perc_12,ortg_12,drtg_12,date_13,ts_perc_13,efg_perc_13,3par_13,ftr_13,orb_perc_13,drb_perc_13,trb_perc_13,ast_perc_13,stl_perc_13,blk_perc_13,tov_perc_13,usg_perc_13,ortg_13,drtg_13,date_14,ts_perc_14,efg_perc_14,3par_14,ftr_14,orb_perc_14,drb_perc_14,trb_perc_14,ast_perc_14,stl_perc_14,blk_perc_14,tov_perc_14,usg_perc_14,ortg_14,drtg_14,date_15,ts_perc_15,efg_perc_15,3par_15,ftr_15,orb_perc_15,drb_perc_15,trb_perc_15,ast_perc_15,stl_perc_15,blk_perc_15,tov_perc_15,usg_perc_15,ortg_15,drtg_15,season
32134,2019-01-01,Detroit Pistons,Milwaukee Bucks,Detroit Pistons,12,0.484,0.446,0.359,0.228,26.9,85.3,50.0,65.7,7.2,3.8,12.1,100.0,100.3,123.9,"(2018-12-30 00:00:00, 2018-12-28 00:00:00, 201...",2018-12-30,2018-12-28,2018-12-26,2018-12-23,2018-12-21,2018-12-19,2018-12-17,2018-12-15,2018-12-12,2018-12-10,2018-12-09,2018-12-07,2018-12-05,2018-12-03,2018-12-01,2018-12-30,0.524,0.467,0.337,0.25,32.1,79.1,53.1,53.8,6.3,3.3,9.7,100.0,111.7,113.8,2018-12-28,0.457,0.423,0.396,0.132,30.8,80.0,50.6,73.5,7.0,8.8,18.6,100.0,88.1,125.2,2018-12-26,0.6,0.55,0.463,0.238,30.6,85.4,59.7,67.6,8.3,8.0,20.7,100.0,109.5,98.1,2018-12-23,0.501,0.47,0.349,0.325,11.4,70.6,43.2,61.8,5.9,13.2,14.4,100.0,93.4,96.3,2018-12-21,0.469,0.439,0.415,0.268,27.3,72.5,48.8,65.6,8.9,9.8,13.2,100.0,95.5,108.9,2018-12-19,0.607,0.596,0.511,0.298,17.4,76.6,47.3,56.5,2.8,2.9,10.9,100.0,119.3,113.7,2018-12-17,0.55,0.523,0.398,0.17,23.8,87.5,54.9,64.1,3.1,3.2,14.5,100.0,106.3,109.4,2018-12-15,0.592,0.55,0.375,0.438,23.8,76.0,52.2,64.1,9.8,11.5,15.9,100.0,110.4,101.6,2018-12-12,0.562,0.541,0.395,0.244,21.7,86.7,53.8,57.5,4.1,17.5,12.8,100.0,108.4,109.4,2018-12-10,0.547,0.542,0.25,0.25,23.8,69.2,48.9,56.1,5.1,9.8,13.9,100.0,104.4,118.8,2018-12-09,0.497,0.485,0.296,0.245,30.4,78.9,50.0,65.1,8.1,10.2,8.4,100.0,109.3,117.4,2018-12-07,0.527,0.455,0.315,0.416,37.8,72.7,55.1,59.5,10.0,5.2,13.2,100.0,111.1,117.1,2018-12-05,0.445,0.416,0.4,0.2,22.2,90.7,52.6,54.3,5.6,2.4,16.2,100.0,86.4,108.0,2018-12-03,0.405,0.362,0.34,0.202,16.4,84

In [66]:
# Calculate z-score
def z_score(value, mean, std):
    return (value - mean) / std

In [67]:
# Season average for each stat
season_avgs = df.groupby(['team', 'season']).aggregate(['mean', 'std']).reset_index()
season_avgs.columns = ["_".join([tup[0], tup[1]]) if len(tup[1]) > 0 else tup[0] for tup in season_avgs.columns]
last_15_games = pd.merge(last_15_games, season_avgs, left_on=['team', 'season'], right_on=['team', 'season'])

In [68]:
stats = ['ts_perc', 'efg_perc', '3par', 'ftr', 'orb_perc', 'drb_perc', 'trb_perc', 
         'ast_perc', 'stl_perc', 'blk_perc', 'tov_perc', 'usg_perc', 'ortg', 'drtg']

for stat in stats:
    # 3 day trend
    last_15_games[stat + '_trend_3'] = \
        (z_score(last_15_games[stat + '_1'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_2'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_3'], last_15_games[stat + '_mean'], last_15_games[stat + '_std'])) / 3
    
    # 5 day trend
    last_15_games[stat + '_trend_5'] = \
        (z_score(last_15_games[stat + '_1'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_2'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_3'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_4'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_5'], last_15_games[stat + '_mean'], last_15_games[stat + '_std'])) / 5
    
    # 10 day trend
    last_15_games[stat + '_trend_10'] = \
        (z_score(last_15_games[stat + '_1'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_2'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_3'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_4'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_5'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_6'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_7'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_8'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_9'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_10'], last_15_games[stat + '_mean'], last_15_games[stat + '_std'])) / 10

### Last Performance

In [82]:
stats = ['ts_perc', 'efg_perc', '3par', 'ftr', 'orb_perc', 'drb_perc', 'trb_perc', 
         'ast_perc', 'stl_perc', 'blk_perc', 'tov_perc', 'usg_perc', 'ortg', 'drtg']
dates = ['_1']
trends = ['_trend_3', '_trend_5', '_trend_10']
cols = ['date', 'visitor', 'home', 'team', 'target'] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, dates))] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, trends))]

last_game = last_15_games[cols].copy()

for stat in stats:
    last_game[stat] = 0
    for date in dates:
        last_game[stat] = last_game[stat] + last_game[stat + date]
    
    last_game[stat] = last_game[stat] / len(dates)

last_game = last_game.groupby(['date', 'visitor', 'home']).aggregate(['mean', 'sum'])

last_game_cols = [col 
                  for col in last_game.columns
                  if ('_perc' in col[0] and 'mean' == col[1] and 'usg' not in col[0]) or ('_perc' not in col[0] and 'sum' == col[1])]

last_game = last_game[last_game_cols].dropna(axis=0)
last_game.columns = [col[0] for col in last_game.columns]
last_game.sample(5)

,,,target,ts_perc_1,efg_perc_1,3par_1,ftr_1,orb_perc_1,drb_perc_1,trb_perc_1,ast_perc_1,stl_perc_1,blk_perc_1,tov_perc_1,ortg_1,drtg_1,ts_perc_trend_3,ts_perc_trend_5,ts_perc_trend_10,efg_perc_trend_3,efg_perc_trend_5,efg_perc_trend_10,3par_trend_3,3par_trend_5,3par_trend_10,ftr_trend_3,ftr_trend_5,ftr_trend_10,orb_perc_trend_3,orb_perc_trend_5,orb_perc_trend_10,drb_perc_trend_3,drb_perc_trend_5,drb_perc_trend_10,trb_perc_trend_3,trb_perc_trend_5,trb_perc_trend_10,ast_perc_trend_3,ast_perc_trend_5,ast_perc_trend_10,stl_perc_trend_3,stl_perc_trend_5,stl_perc_trend_10,blk_perc_trend_3,blk_perc_trend_5,blk_perc_trend_10,tov_perc_trend_3,tov_perc_trend_5,tov_perc_trend_10,ortg_trend_3,ortg_trend_5,ortg_trend_10,drtg_trend_3,drtg_trend_5,drtg_trend_10,ts_perc,efg_perc,3par,ftr,orb_perc,drb_perc,trb_perc,ast_perc,stl_perc,blk_perc,tov_perc,ortg,drtg
date,visitor,home,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-12-22,Philadelphia 76ers,Boston Celtics,8,0.506,0.4610,0.372,0.525,20.45,78.55,49.60,51.25,8.45,4.10,17.15,184.8,222.4,-0.323082,-0.128941,0.164240,-0.427135,-0.279028,0.110986,0.191658,0.675479,0.825326,-0.837445,-0.417409,0.133947,0.095401,0.051987,-0.074003,-0.006501,-0.389928,0.332930,-0.148949,-0.259149,0.318153,-0.433322,-0.230646,0.040292,0.654950,0.478944,-0.018553,-0.462122,0.274447,0.314756,0.323017,0.170461,0.053154,-0.819827,-0.260052,0.289365,0.574962,-0.234853,-0.638438,0.506,0.4610,0.372,0.525,20.45,78.55,49.60,51.25,8.45,4.10,17.15,184.8,222.4
2011-01-07,Cleveland Cavaliers,Golden State Warriors,18,0.611,0.5565,0.531,0.888,20.75,64.75,43.25,69.40,8.20,5.35,11.65,236.2,245.2,0.352396,0.236084,0.194289,0.333606,0.221040,0.209276,0.652481,1.137386,1.136336,0.019385,0.132308,0.098880,-0.458685,-0.465633,-0.447791,0.007501,0.038802,-0.070537,-0.303537,-0.171750,-0.252568,0.108352,-0.047691,0.390626,-0.153232,0.137091,-0.014517,-0.487930,-0.088419,0.018452,0.004958,0.238188,0.015268,0.285234,-0.051725,0.094422,0.912710,0.522390,-0.058927,0.611,0.5565,0.531,0.888,20.75,64.75,43.25,69.40,8.20,5.35,11.65,236.2,245.2
2020-08-09,Brooklyn Nets,Los Angeles Clippers,33,0.616,0.5845,0.865,0.655,18.95,77.00,49.55,64.25,7.05,5.20,10.50,243.3,225.1,0.087528,-0.257840,-0.380319,0.133796,-0.288006,-0.445927,-0.829836,-0.786952,-1.071395,-0.715156,0.115249,0.273882,-0.200955,0.139808,0.356005,-0.030643,-0.082522,-1.156576,-0.234771,-0.245547,-0.573911,-0.191534,0.078092,-0.005961,-0.255089,0.046081,0.179507,0.105235,0.013492,-0.324995,-0.485674,-0.339875,-0.075185,0.336926,-0.238926,-0.460988,0.455248,0.066721,0.228320,0.616,0.5845,0.865,0.655,18.95,77.00,49.55,64.25,7.05,5.20,10.50,243.3,225.1
2009-04-05,Phoenix Suns,Dallas Mavericks,21,0.610,0.5800,0.560,0.651,24.80,73.40,49.15,52.25,6.10,11.20,9.40,253.2,231.4,0.436023,-0.388383,0.053822,0.647116,-0.173569,0.176208,0.889797,0.555133,0.330566,-0.047791,-0.428821,-0.052492,-0.105758,0.129501,0.185434,0.005351,-0.002991,-0.047064,-0.029446,-0.144834,0.119750,-0.038868,-0.031408,0.116725,0.208012,0.107939,0.079677,0.385577,0.318600,0.382146,-0.436391,-0.548956,-0.618548,1.247896,-0.014185,0.711257,0.374847,0.559896,0.502637,0.610,0.5800,0.560,0.651,24.80,73.40,49.15,52.25,6.10,11.20,9.40,253.2,231.4
2015-03-08,Charlotte Hornets,Detroit Pistons,9,0.487,0.4460,0.519,0.402,30.00,80.25,55.60,59.55,7.55,7.55,11.85,200.2,200.6,0.115833,0.194387,0.148649,0.225448,0.262116,0.161665,-0.879287,-0.347630,-0.143553,-0.528724,-0.071613,-0.031803,0.306726,0.025157,-0.020333,-0.001507,0.032998,-0.224584,0.535916,0.382151,0.011974,0.296057,-0.012679,-0.020911,0.228477,-0.123136,0.049658,-0.469924,-0.193956,-0.316276,0.499405,0.633117,0.197758,-0.020900,-0.080550,0.148483,-0.897504,-0.608813,-0.269860,0.487,0.4460,0.519,0.402,30.00,80.25,55.60,59.55,7.55,7.55,11.85,200.2,200.6


### Last 5 Performances

In [83]:
stats = ['ts_perc', 'efg_perc', '3par', 'ftr', 'orb_perc', 'drb_perc', 'trb_perc', 
         'ast_perc', 'stl_perc', 'blk_perc', 'tov_perc', 'usg_perc', 'ortg', 'drtg']
dates = ['_1', '_2', '_3', '_4', '_5']
trends = ['_trend_3', '_trend_5', '_trend_10']
cols = ['date', 'visitor', 'home', 'team', 'target'] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, dates))] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, trends))]

last_5_game = last_15_games[cols].copy()

for stat in stats:
    last_5_game[stat] = 0
    for date in dates:
        last_5_game[stat] = last_5_game[stat] + last_5_game[stat + date]
    
    last_5_game[stat] = last_5_game[stat] / len(dates)

last_5_game = last_5_game.groupby(['date', 'visitor', 'home']).aggregate(['mean', 'sum'])

last_5_game_cols = [col 
                    for col in last_5_games.columns
                    if ('_perc' in col[0] and 'mean' == col[1] and 'usg' not in col[0]) or ('_perc' not in col[0] and 'sum' == col[1])]

last_5_games = last_5_games[last_5_game_cols].dropna(axis=0)
last_5_games.columns = [col[0] for col in last_5_games.columns]
last_5_games.head()

Empty DataFrame
Columns: []
Index: [(2006-11-08 00:00:00, Los Angeles Lakers, Portland Trail Blazers), (2006-11-09 00:00:00, Dallas Mavericks, Phoenix Suns), (2006-11-09 00:00:00, New Orleans/Oklahoma City Hornets, Golden State Warriors), (2006-11-10 00:00:00, Denver Nuggets, Philadelphia 76ers), (2006-11-10 00:00:00, Detroit Pistons, Los Angeles Lakers)]

### Last 10 Perfomances

In [84]:
stats = ['ts_perc', 'efg_perc', '3par', 'ftr', 'orb_perc', 'drb_perc', 'trb_perc', 
         'ast_perc', 'stl_perc', 'blk_perc', 'tov_perc', 'usg_perc', 'ortg', 'drtg']
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10']
trends = ['_trend_3', '_trend_5', '_trend_10']
cols = ['date', 'visitor', 'home', 'team', 'target'] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, dates))] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, trends))]

last_10_game = last_15_games[cols].copy()

for stat in stats:
    last_10_game[stat] = 0
    for date in dates:
        last_10_game[stat] = last_10_game[stat] + last_10_game[stat + date]
    
    last_10_game[stat] = last_10_game[stat] / len(dates)

last_10_game = last_10_game.groupby(['date', 'visitor', 'home']).aggregate(['mean', 'sum'])

last_10_game_cols = [col 
                    for col in last_10_games.columns
                    if ('_perc' in col[0] and 'mean' == col[1] and 'usg' not in col[0]) or ('_perc' not in col[0] and 'sum' == col[1])]

last_10_games = last_10_games[last_10_game_cols].dropna(axis=0)
last_10_games.columns = [col[0] for col in last_10_games.columns]
last_10_games.head()

Empty DataFrame
Columns: []
Index: [(2006-11-18 00:00:00, Boston Celtics, New York Knicks), (2006-11-18 00:00:00, Portland Trail Blazers, New Jersey Nets), (2006-11-18 00:00:00, Seattle SuperSonics, Golden State Warriors), (2006-11-20 00:00:00, Houston Rockets, New York Knicks), (2006-11-20 00:00:00, New Jersey Nets, Seattle SuperSonics)]

### Last 15 Performances (Unweighted)

In [85]:
stats = ['ts_perc', 'efg_perc', '3par', 'ftr', 'orb_perc', 'drb_perc', 'trb_perc', 
         'ast_perc', 'stl_perc', 'blk_perc', 'tov_perc', 'usg_perc', 'ortg', 'drtg']
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
trends = ['_trend_3', '_trend_5', '_trend_10']
cols = ['date', 'visitor', 'home', 'team', 'target'] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, dates))] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, trends))]

last_15_games_unweighted = last_15_games[cols].copy()

for stat in stats:
    last_15_games_unweighted[stat] = 0
    for date in dates:
        last_15_games_unweighted[stat] = last_15_games_unweighted[stat] + last_15_games_unweighted[stat + date]
    
    last_15_games_unweighted[stat] = last_15_games_unweighted[stat] / len(dates)

last_15_games_unweighted = last_15_games_unweighted.groupby(['date', 'visitor', 'home']).aggregate(['mean', 'sum'])

last_15_game_cols = [col 
                     for col in last_15_games_unweighted.columns
                     if ('_perc' in col[0] and 'mean' == col[1] and 'usg' not in col[0]) or ('_perc' not in col[0] and 'sum' == col[1])]

last_15_games_unweighted = last_15_games_unweighted[last_15_game_cols].dropna(axis=0)
last_15_games_unweighted.columns = [col[0] for col in last_15_games_unweighted.columns]
last_15_games_unweighted.head()

target  ts_perc_1  \
date       visitor         home                                        
2006-11-28 Indiana Pacers  Portland Trail Blazers      13     0.5780   
           New York Knicks Chicago Bulls                9     0.5290   
2006-11-29 Indiana Pacers  Golden State Warriors       18     0.6105   
           New York Knicks Cleveland Cavaliers         13     0.5170   
           Orlando Magic   Seattle SuperSonics          8     0.4515   

                                                   ts_perc_2  ts_perc_3  \
date       visitor         home                                           
2006-11-28 Indiana Pacers  Portland Trail Blazers     0.5330      0.554   
           New York Knicks Chicago Bulls              0.5930      0.522   
2006-11-29 Indiana Pacers  Golden State Warriors      0.4685      0.568   
           New York Knicks Cleveland Cavaliers        0.5290      0.593   
           Orlando Magic   Seattle SuperSonics        0.5420      0.532   

                                                   ts_perc_4  ts_perc_5  \
date       visitor         home                                           
2006-11-28 Indiana Pacers  Portland Trail Blazers      0.578     0.5650   
           New York Knicks Chicago Bulls               0.464     0.5460   
2006-11-29 Indiana Pacers  Golden State Warriors       0.516     0.5450   
           New York Knicks Cleveland Cavaliers         0.522     0.4640   
           Orlando Magic   Seattle SuperSonics         0.548     0.5025   

                                                   ts_perc_6  ts_perc_7  \
date       visitor         home                                           
2006-11-28 Indiana Pacers  Portland Trail Blazers      0.517     0.4870   
           New York Knicks Chicago Bulls               0.592     0.5350   
2006-11-29 Indiana Pacers  Golden State Warriors       0.542     0.5670   
           New York Knicks Cleveland Cavaliers         0.546     0.5920   
           Orlando Magic   Seattle SuperSonics         0.591     0.5545   

                                                   ts_perc_8  ts_perc_9  \
date       visitor         home                                           
2006-11-28 Indiana Pacers  Portland Trail Blazers      0.514      0.560   
           New York Knicks Chicago Bulls               0.546      0.514   
2006-11-29 Indiana Pacers  Golden State Warriors       0.497      0.595   
           New York Knicks Cleveland Cavaliers         0.535      0.546   
           Orlando Magic   Seattle SuperSonics         0.567      0.555   

                                                   ts_perc_10  ts_perc_11  \
date       visitor         home                                             
2006-11-28 Indiana Pacers  Portland Trail Blazers      0.5440      0.6120   
           New York Knicks Chicago Bulls               0.5110      0.5930   
2006-11-29 Indiana Pacers  Golden State Warriors       0.5780      0.5040   
           New York Knicks Cleveland Cavaliers         0.5140      0.5110   
           Orlando Magic   Seattle SuperSonics         0.5305      0.5545   

                                                   ts_perc_12  ts_perc_13  \
date       visitor         home                                             
2006-11-28 Indiana Pacers  Portland Trail Blazers      0.5090      0.4890   
           New York Knicks Chicago Bulls               0.5240      0.5150   
2006-11-29 Indiana Pacers  Golden State Warriors       0.5755      0.5255   
           New York Knicks Cleveland Cavaliers         0.5930      0.5240   
           Orlando Magic   Seattle SuperSonics         0.5675      0.5600   

                                                   ts_perc_14  ts_perc_15  \
date       visitor         home                                             
2006-11-28 Indiana Pacers  Portland Trail Blazers      0.5400      0.6220   
           New York Knicks Chicago Bulls               0.5290      0.5320   
2006-11-29 Indiana Pacers  Golden State Warriors      

### Last 15 Performances (Weighted)

In [86]:
stats = ['ts_perc', 'efg_perc', '3par', 'ftr', 'orb_perc', 'drb_perc', 'trb_perc', 
         'ast_perc', 'stl_perc', 'blk_perc', 'tov_perc', 'usg_perc', 'ortg', 'drtg']
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
trends = ['_trend_3', '_trend_5', '_trend_10']
cols = ['date', 'visitor', 'home', 'team', 'target'] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, dates))] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, trends))]

last_15_games_weighted = last_15_games[cols].copy()

for stat in stats:
    last_15_games_weighted[stat] = 0
    for date in dates:
        weight = ((int(date.strip('_')) - 1) // 5) + 1
        last_15_games_weighted[stat] = (last_15_games_weighted[stat] + last_15_games_weighted[stat + date]) * weight
    
    last_15_games_weighted[stat] = last_15_games_weighted[stat] / len(dates)

last_15_games_weighted = last_15_games_weighted.groupby(['date', 'visitor', 'home']).aggregate(['mean', 'sum'])

last_15_game_cols = [col 
                     for col in last_15_games_weighted.columns
                     if ('_perc' in col[0] and 'mean' == col[1] and 'usg' not in col[0]) or ('_perc' not in col[0] and 'sum' == col[1])]

last_15_games_weighted = last_15_games_weighted[last_15_game_cols].dropna(axis=0)
last_15_games_weighted.columns = [col[0] for col in last_15_games_weighted.columns]
last_15_games_weighted.sample(5)

,,,target,ts_perc_1,ts_perc_2,ts_perc_3,ts_perc_4,ts_perc_5,ts_perc_6,ts_perc_7,ts_perc_8,ts_perc_9,ts_perc_10,ts_perc_11,ts_perc_12,ts_perc_13,ts_perc_14,ts_perc_15,efg_perc_1,efg_perc_2,efg_perc_3,efg_perc_4,efg_perc_5,efg_perc_6,efg_perc_7,efg_perc_8,efg_perc_9,efg_perc_10,efg_perc_11,efg_perc_12,efg_perc_13,efg_perc_14,efg_perc_15,3par_1,3par_2,3par_3,3par_4,3par_5,3par_6,3par_7,3par_8,3par_9,3par_10,3par_11,3par_12,3par_13,3par_14,3par_15,ftr_1,ftr_2,ftr_3,ftr_4,ftr_5,ftr_6,ftr_7,ftr_8,ftr_9,ftr_10,ftr_11,ftr_12,ftr_13,ftr_14,ftr_15,orb_perc_1,orb_perc_2,orb_perc_3,orb_perc_4,orb_perc_5,orb_perc_6,orb_perc_7,orb_perc_8,orb_perc_9,orb_perc_10,orb_perc_11,orb_perc_12,orb_perc_13,orb_perc_14,orb_perc_15,drb_perc_1,drb_perc_2,drb_perc_3,drb_perc_4,drb_perc_5,drb_perc_6,drb_perc_7,drb_perc_8,drb_perc_9,drb_perc_10,drb_perc_11,drb_perc_12,drb_perc_13,drb_perc_14,drb_perc_15,trb_perc_1,trb_perc_2,trb_perc_3,trb_perc_4,trb_perc_5,trb_perc_6,trb_perc_7,trb_perc_8,trb_perc_9,trb_perc_10,trb_perc_11,trb_perc_12,trb_perc_13,trb_perc_14,trb_perc_15,ast_perc_1,ast_perc_2,ast_perc_3,ast_perc_4,ast_perc_5,ast_perc_6,ast_perc_7,ast_perc_8,ast_perc_9,ast_perc_10,ast_perc_11,ast_perc_12,ast_perc_13,ast_perc_14,ast_perc_15,stl_perc_1,stl_perc_2,stl_perc_3,stl_perc_4,stl_perc_5,stl_perc_6,stl_perc_7,stl_perc_8,stl_perc_9,stl_perc_10,stl_perc_11,stl_perc_12,stl_perc_13,stl_perc_14,stl_perc_15,blk_perc_1,blk_perc_2,blk_perc_3,blk_perc_4,blk_perc_5,blk_perc_6,blk_perc_7,blk_perc_8,blk_perc_9,blk_perc_10,blk_perc_11,blk_perc_12,blk_perc_13,blk_perc_14,blk_perc_15,tov_perc_1,tov_perc_2,tov_perc_3,tov_perc_4,tov_perc_5,tov_perc_6,tov_perc_7,tov_perc_8,tov_perc_9,tov_perc_10,tov_perc_11,tov_perc_12,tov_perc_13,tov_perc_14,tov_perc_15,ortg_1,ortg_2,ortg_3,ortg_4,ortg_5,ortg_6,ortg_7,ortg_8,ortg_9,ortg_10,ortg_11,ortg_12,ortg_13,ortg_14,ortg_15,drtg_1,drtg_2,drtg_3,drtg_4,drtg_5,drtg_6,drtg_7,drtg_8,drtg_9,drtg_10,drtg_11,drtg_12,drtg_13,drtg_14,drtg_15,ts_perc_trend_3,ts_perc_trend_5,ts_perc_trend_10,efg_perc_trend_3,efg_perc_trend_5,efg_perc_trend_10,3par_trend_3,3par_trend_5,3par_trend_10,ftr_trend_3,ftr_trend_5,ftr_trend_10,orb_perc_trend_3,orb_perc_trend_5,orb_perc_trend_10,drb_perc_trend_3,drb_perc_trend_5,drb_perc_trend_10,trb_perc_trend_3,trb_perc_trend_5,trb_perc_trend_10,ast_perc_trend_3,ast_perc_trend_5,ast_perc_trend_10,stl_perc_trend_3,stl_perc_trend_5,stl_perc_trend_10,blk_perc_trend_3,blk_perc_trend_5,blk_perc_trend_10,tov_perc_trend_3,tov_perc_trend_5,tov_perc_trend_10,ortg_trend_3,ortg_trend_5,ortg_trend_10,drtg_trend_3,drtg_trend_5,drtg_trend_10,ts_perc,efg_perc,3par,ftr,orb_perc,drb_perc,trb_perc,ast_perc,stl_perc,blk_perc,tov_perc,ortg,drtg
date,visitor,home,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2009-11-29,Atlanta Hawks,Detroit Pistons,8,0.5405,0.4530,0.4585,0.5600,0.4895,0.5250,0.5485,0.5360,0.6200,0.5560,0.5080,0.5660,0.5100,0.5330,0.5130,0.5070,0.4265,0.4300,0.513,0.4425,0.4745,0.5275,0.494,0.5850,0.5130,0.4755,0.5350,0.4345,0.477,0.4245,0.430,0.401,0.436,0.444,0.413,0.365,0.384,0.414,0.354,0.416,0.342,0.393,0.271,0.483,0.499,0.512,0.474,0.316,0.646,0.540,0.595,0.535,0.669,0.532,0.604,0.777,0.525,0.803,0.644,0.922,30.75,30.8,36.70,33.10,29.95,26.50,34.70,37.90,23.60,34.60,16.85,29.75,25.85,29.30,22.80,85.35,70.45,71.05,67.00,78.75,72.05,71.65,76.05,74.85,73.25,72.50,67.80,77.80,66.10,80.45,55.95,49.35,51.30,51.10,51.30,49.05,53.80,56.2,51.30,55.85,43.20,51.10,53.65,46.95,51.45,51.5,46.60,45.15,47.75,51.20,50.65,43.95,47.95,58.65,52.80,32.25,41.45,43.20,53.65,51.25,6.35,7.55,7.50,9.25,6.85,8.10,7.45,6.20,8.50,6.85,8.25,2.75,6.30,7.40,6.05,4.15,8.95,8.80,11.10,7.95,9.00,6.90,9.20,11.65,10.75,11.60,10.95,10.0,4.65,9.15,12.15,13.00,8.85,14.30,9.65,13.75,12.40,12.05,13.15,10.85,8.95,12.55,10.05,17.75,13.20,226.6,191.6,206.4,223

## Correlations of performances (last 1, last 5, last 10)

In [87]:
corr_df = pd.DataFrame()

# Correlations for last 15 game stats vs 3pt made (unweighted)
for col in last_15_games_weighted:
    corr_p = pearsonr(last_15_games_weighted['target'], last_15_games_weighted[col])
    row = {'last': '15_weighted', 'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)

# Correlations for last 15 game stats vs 3pt made (unweighted)
for col in last_15_games_unweighted:
    corr_p = pearsonr(last_15_games_unweighted['target'], last_15_games_unweighted[col])
    row = {'last': '15_unweighted', 'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)

# Correlations for last 10 game stats vs 3pt made 
for col in last_10_games:
    corr_p = pearsonr(last_10_games['target'], last_10_games[col])
    row = {'last': 10, 'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)
    
# Correlations for last 10 game stats vs 3pt made 
for col in last_5_games:
    corr_p = pearsonr(last_5_games['target'], last_5_games[col])
    row = {'last': 5, 'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)
    
# Correlations for last game stats vs 3pt made 
for col in last_game:
    corr_p = pearsonr(last_game['target'], last_game[col])
    row = {'last': 1, 'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)
    
# Print each correlation
for stat in stats + [tup[0] + tup[1] for tup in list(itertools.product(stats, trends))]:
    print(f'Stat: {stat}')
    print(corr_df[(corr_df['stat'] == stat) & (corr_df['p-value'] < .05)].set_index(['last']).drop(['stat'], axis=1))
    print('\n')

Stat: ts_perc
               corr  p-value
last                        
15_weighted    0.38      0.0
15_unweighted  0.42      0.0
1              0.20      0.0


Stat: efg_perc
               corr  p-value
last                        
15_weighted    0.44      0.0
15_unweighted  0.48      0.0
1              0.24      0.0


Stat: 3par
               corr  p-value
last                        
15_weighted    0.71      0.0
15_unweighted  0.71      0.0
1              0.64      0.0


Stat: ftr
               corr  p-value
last                        
15_weighted   -0.29      0.0
15_unweighted -0.32      0.0
1             -0.16      0.0


Stat: orb_perc
               corr  p-value
last                        
15_weighted   -0.42      0.0
15_unweighted -0.46      0.0
1             -0.23      0.0


Stat: drb_perc
               corr  p-value
last                        
15_weighted    0.37      0.0
15_unweighted  0.43      0.0
1              0.18      0.0


Stat: trb_perc
               corr  p-